# דוגמה לסוכן הזמנת מלון וטיסות

הפתרון הזה יעזור לך להזמין כרטיסי טיסה ומלון. התרחיש הוא טיול מלונדון הית'רו LHR ב-20 בפברואר 2024 לניו יורק JFK עם חזרה ב-27 בפברואר 2025, טיסה במחלקת תיירים עם בריטיש איירווייז בלבד. אני רוצה לשהות במלון הילטון בניו יורק, בבקשה ספק עלויות עבור הטיסה והמלון.


# אתחול שירות Azure AI Agent וקבלת מידע על תצורה מתוך **.env**

### **.env**

צרו קובץ .env

**.env** מכיל את מחרוזת החיבור של שירות Azure AI Agent, המודל שבו משתמש AOAI, ואת שירות ה-API של Google API Search, ENDPOINT, ועוד.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "שם פריסת המודל של שירות Azure AI Agent שלכם"

[**NOTE**] תצטרכו מודל עם מגבלת קצב של 100,000 (אסימונים לדקה) ומגבלת קצב של 600 (בקשות לדקה).

  ניתן להשיג מודל ב-Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "מחרוזת החיבור של פרויקט שירות Azure AI Agent שלכם"

  ניתן להשיג את מחרוזת החיבור של הפרויקט במסך סקירת הפרויקט בפורטל AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "מפתח ה-API של SERPAPI שלכם"
- **SERPAPI_SEARCH_ENDPOINT** = "נקודת הקצה של SERPAPI שלכם"

כדי להשיג את שם פריסת המודל ומחרוזת החיבור של פרויקט שירות Azure AI Agent, עליכם ליצור את שירות Azure AI Agent. מומלץ להשתמש [בתבנית הזו](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) כדי ליצור אותו ישירות. (***הערה:*** שירות Azure AI Agent מוגבל כרגע לאזורים מסוימים. מומלץ לעיין [בקישור הזה](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) כדי להגדיר את האזור).

הסוכן צריך גישה ל-SERPAPI. מומלץ להירשם באמצעות [הקישור הזה](https://serpapi.com/searches). לאחר ההרשמה, תוכלו לקבל מפתח API ייחודי ונקודת קצה (ENDPOINT).


# התקנה

כדי להפעיל את המחברת הזו, יש לוודא שהתקנתם את הספריות הנדרשות על ידי הרצת `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

גרסת ה-Semantic Kernel שלך צריכה להיות לפחות 1.27.2.


טען את קובץ .env שלך והגדרות המשאבים, אנא ודא שהוספת את המפתחות וההגדרות שלך ויצרת קובץ .env מקומי.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# התחברות ל-Azure

כעת עליך להתחבר ל-Azure. פתח מסוף והרץ את הפקודה הבאה:

```bash
az login
```

פקודה זו תבקש ממך להזין את פרטי ההתחברות שלך ל-Azure, מה שיאפשר לשירות Azure AI Agent לפעול בצורה תקינה.


# הסבר:
זוהי משתנה שמאחסן את מפתח ה-API לצורך גישה לשירות SERP (דף תוצאות של מנוע חיפוש). מפתח API הוא מזהה ייחודי המשמש לאימות בקשות הקשורות לחשבון שלך.

מטרה: המטרה של שורה זו היא לאחסן את מפתח ה-API במשתנה כך שניתן יהיה להשתמש בו לאימות בקשות לשירות ה-SERP API. מפתח ה-API נדרש כדי לגשת לשירות ולבצע חיפושים.

כיצד להשיג מפתח SERP API: כדי להשיג מפתח SERP API, בצע את השלבים הכלליים הבאים באתר https://serpapi.com (ייתכן שהשלבים המדויקים ישתנו בהתאם לשירות ה-SERP API הספציפי שבו אתה משתמש):

בחר שירות SERP API: קיימים מספר שירותי SERP API, כמו SerpAPI, Google Custom Search JSON API ואחרים. בחר את השירות שמתאים ביותר לצרכים שלך.

הירשם לחשבון: עבור לאתר של שירות ה-SERP API שבחרת והירשם לחשבון. ייתכן שתצטרך לספק מידע בסיסי ולאמת את כתובת הדוא"ל שלך.

צור מפתח API: לאחר ההרשמה, היכנס לחשבונך ונווט אל החלק של ה-API או לוח הבקרה. חפש אפשרות ליצירת מפתח API חדש או להפקתו.
העתק את מפתח ה-API לקובץ ה-.env שלך.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# הסבר:
BASE_URL: זו משתנה שמאחסן את כתובת הבסיס עבור נקודת הקצה של SERP API. שם המשתנה BASE_URL הוא מוסכמה שמצביעה על כך שכתובת זו היא נקודת ההתחלה לביצוע בקשות API.
'https://serpapi.com/search':

זו מחרוזת הכתובת בפועל שמוקצת למשתנה BASE_URL. היא מייצגת את נקודת הקצה לביצוע שאילתות חיפוש באמצעות SERP API.

# מטרה:
המטרה של שורה זו היא להגדיר קבוע שמחזיק את כתובת הבסיס עבור SERP API. כתובת זו תשמש כנקודת ההתחלה לבניית בקשות API לביצוע פעולות חיפוש.

# שימוש:
על ידי הגדרת כתובת הבסיס במשתנה, ניתן להשתמש בה בקלות בכל הקוד בכל פעם שיש צורך לבצע בקשות ל-SERP API. זה הופך את הקוד ליותר ניתן לתחזוקה ומפחית את הסיכון לטעויות שנובעות מהקשחת הכתובת במקומות שונים. הדוגמה הנוכחית היא https://serpapi.com/search?engine=bing שמשתמשת ב-Bing search API. ניתן לבחור API שונה בכתובת https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# הסבר:

כאן נמצא קוד התוסף שלך.

הגדרת מחלקה: `class BookingPlugin`: מגדירה מחלקה בשם BookingPlugin שמכילה שיטות להזמנת מלונות וטיסות.

שיטת הזמנת מלון:

- `@kernel_function(description="booking hotel")`: דקורטור שמתאר את הפונקציה כפונקציה מרכזית להזמנת מלונות.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: מגדירה שיטה להזמנת מלונות עם פרמטרים מסומנים וסוג החזרה.

השיטה יוצרת מילון של פרמטרים לבקשת הזמנת המלון ושולחת בקשת GET ל-SERP API. היא בודקת את סטטוס התגובה ומחזירה את מאפייני המלון אם הבקשה הצליחה, או None אם הבקשה נכשלה.

שיטת הזמנת טיסה:

- `@kernel_function(description="booking flight")`: דקורטור שמתאר את הפונקציה כפונקציה מרכזית להזמנת טיסות.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: מגדירה שיטה להזמנת טיסות עם פרמטרים מסומנים וסוג החזרה.

השיטה יוצרת מילונים של פרמטרים לבקשות הטיסה היוצאת והחוזרת ושולחת בקשות GET ל-SERP API. היא בודקת את סטטוס התגובה ומוסיפה את מידע הטיסה למחרוזת התוצאה אם הבקשה הצליחה, או מדפיסה הודעת שגיאה אם הבקשה נכשלה. השיטה מחזירה את מחרוזת התוצאה שמכילה את מידע הטיסה.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# הסבר:
פקודות ייבוא: ייבוא מודולים נחוצים עבור אישורי Azure, סוכן AI, תוכן הודעות צ'אט, תפקיד מחבר, ודקורטור פונקציות ליבה.

מנהל הקשר אסינכרוני: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: מגדיר מנהל הקשר אסינכרוני לטיפול באישורי Azure וליצירת לקוח סוכן AI.

שם הסוכן והוראות:
- `AGENT_NAME = "BookingAgent"`: מגדיר את שם הסוכן.
- `AGENT_INSTRUCTIONS = """..."""`: מספק הוראות מפורטות לסוכן כיצד לטפל בבקשות הזמנה.

יצירת הגדרת סוכן: `agent_definition = await client.agents.create_agent(...)`: יוצר הגדרת סוכן עם המודל, השם וההוראות שצוינו.

יצירת סוכן AzureAI: `agent = AzureAIAgent(...)`: יוצר סוכן AzureAI באמצעות הלקוח, הגדרת הסוכן והתוסף שהוגדר.

יצירת שרשור: `thread: AzureAIAgentThread | None = None`: יוצר שרשור עבור הסוכן. אין חובה ליצור שרשור מראש - אם הערך `None` מסופק, שרשור חדש ייווצר במהלך הקריאה הראשונה ויוחזר כחלק מהתגובה.

קלטי משתמש: `user_inputs = ["..."]`: מגדיר רשימה של קלטי משתמש שהסוכן צריך לעבד.

בבלוק `finally`, מוחקים את השרשור והסוכן כדי לשחרר משאבים.


# אימות

המחלקה `DefaultAzureCredential` היא חלק מ-SDK של Azure עבור Python. היא מספקת דרך ברירת מחדל לאמת מול שירותי Azure. המחלקה מנסה לאמת באמצעות מספר שיטות בסדר מסוים, כמו משתני סביבה, זהות מנוהלת ונתוני אימות של Azure CLI.

פעולות אסינכרוניות: המודול aio מציין שהמחלקה DefaultAzureCredential תומכת בפעולות אסינכרוניות. המשמעות היא שניתן להשתמש בה עם asyncio כדי לבצע בקשות אימות שאינן חוסמות.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). למרות שאנו שואפים לדיוק, יש לקחת בחשבון שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור הסמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
